In [31]:
%matplotlib inline
from matplotlib import pyplot
import numpy as np
import pandas as pd
from datetime import date
import time
import pymongo
client = pymongo.MongoClient()
db = client.customer_database

customern = 100000

with open('CSV_Database_of_Last_Names.csv', 'r') as f:
    last_name_list = [row for row in f.readline().split('\r')][1:]
with open('states.json') as f:
    states = eval(f.readline())

states_data = []
with open('states.csv') as f:
    for row in f:
        r = []
        states_data.append(row.split(','))

names_list_male = []
with open('names.csv') as f:
    names_list_male = [row.split(',')[0] for row in f]
    names_list_male = [unicode(name, errors='ignore') for name in names_list_male if len(name)>1][2:]
    f.seek(0)
    names_list_female = [row.split(',')[3] for row in f]
    names_list_female = [unicode(name, errors='ignore') for name in names_list_female if len(name)>1][2:]

us_pop = sum(float(row[6]) for row in states_data[2:])+1
states_population = {row[4] : float(row[6])/us_pop for row in states_data[2:]}

states = states_population.keys()
mn_onehot_id = np.random.multinomial(1, states_population.values(), customern)
state_id = [int(np.nonzero(row)[0]) for row in mn_onehot_id]
state = [states[state_id_i] for state_id_i in state_id]

gender_by_state_p = np.random.rand(len(state_id))*0.5+0.25
gender_binary = [np.random.binomial(1,gender_by_state_p[state_id_i]) for state_id_i in state_id]
gender = ['male' if g else 'female' for g in gender_binary]

name = [names_list_female[int(np.random.rand() * len(names_list_female))] 
        if gender_i=='female' 
        else names_list_male[int(np.random.rand() * len(names_list_male))]
        for gender_i in gender]
name = [name_i+' '+last_name_list[int(np.random.rand() * len(last_name_list))] for name_i in name]

mean_state_churn_rate = np.random.rand(len(states))*0.5+0.25
var_state_churn_rate = np.random.rand(len(states))+1

churn_rate = np.random.randn(len(state_id))/24
churn_rate = [churn_rate_i * var_state_churn_rate[state_id_i] + mean_state_churn_rate[state_id_i] for churn_rate_i, state_id_i in zip(churn_rate, state_id)]

status_rand = np.random.rand(len(state_id))
status_names = ['platinum', 'gold', 'silver', 'bronze']
status_probs = [0.01, 0.05, 0.1, 0.84]

mn_onehot_id = np.random.multinomial(1, status_probs, len(state_id))
status_id = [int(np.nonzero(row)[0]) for row in mn_onehot_id]
status = [status_names[status_id_i] for status_id_i in status_id]

In [32]:
today = date.today().toordinal()
def gen_survey():
    n_dates = int(abs(np.random.randn()*5))
    date_str = []
    results = []
    if n_dates>0:
        date_rand = start_date+np.array(np.random.rand(n_dates)*(today-start_date), dtype='int')
        date_rand.sort()
        date_str = [date.fromordinal(date_i).isoformat() for date_i in date_rand]

        results = [min(abs(np.random.randn()),1)]
        ratio = np.random.rand()*0.05+0.05
        for i in range(n_dates-1):
            jump = (np.random.rand()>0.7)*np.random.randn()/12
            results.append(max(min(results[-1]*0.75 + ratio*3 + jump + np.random.randn()*0.01,1),0))
        results = [int(result_i*100) for result_i in results]
    return date_str, results

In [33]:
def gen_survey():
    n_dates = int(abs(np.random.randn()*5))
    date_str = []
    results = []
    if n_dates>0:
        date_rand = start_date+np.array(np.random.rand(n_dates)*(today-start_date), dtype='int')
        date_rand.sort()
        date_str = [date.fromordinal(date_i).isoformat() for date_i in date_rand]

        results = [min(abs(np.random.randn()),1)]
        ratio = np.random.rand()*0.05+0.05
        for i in range(n_dates-1):
            jump = (np.random.rand()>0.7)*np.random.randn()/12
            results.append(max(min(results[-1]*0.75 + ratio*3 + jump + np.random.randn()*0.01,1),0))
        results = [int(result_i*100) for result_i in results]
    return date_str, results

In [34]:
start_date = 365*2012
today = date.today().toordinal()
def gen_survey():
    n_dates = int(abs(np.random.randn()*5))
    date_str = []
    results = []
    if n_dates>0:
        date_rand = start_date+np.array(np.random.rand(n_dates)*(today-start_date), dtype='int')
        date_rand.sort()
        date_str = [date.fromordinal(date_i).isoformat() for date_i in date_rand]

        results = [min(abs(np.random.randn()),1)]
        ratio = np.random.rand()*0.05+0.05
        for i in range(n_dates-1):
            jump = (np.random.rand()>0.7)*np.random.randn()/12
            results.append(max(min(results[-1]*0.75 + ratio*3 + jump + np.random.randn()*0.01,1),0))
        results = [int(result_i*100) for result_i in results]
    return date_str, results

In [35]:
surveys = [gen_survey() for i in range(customern)]

surveys_d = [[{'surveyDate':date_i, 'surveyResult':result_i, 'surveyType':'General'} 
             for date_i,result_i in zip(date, result)]
             for (date,result) in surveys]

In [36]:
monthly_profit = [{'yearAndMonth':'2015-01','profitability':303},
                  {'yearAndMonth':'2015-02','profitability':340},
                  {'yearAndMonth':'2015-03','profitability':295},
                  {'yearAndMonth':'2015-04','profitability':293},
                  {'yearAndMonth':'2015-05','profitability':292},
                  {'yearAndMonth':'2015-06','profitability':267},
                  {'yearAndMonth':'2015-07','profitability':348},
                  {'yearAndMonth':'2015-08','profitability':327},
                  {'yearAndMonth':'2015-09','profitability':261},
                  {'yearAndMonth':'2015-10','profitability':257},
                  {'yearAndMonth':'2015-11','profitability':325},
                  {'yearAndMonth':'2015-12','profitability':332},
                  {'yearAndMonth':'2016-01','profitability':277},
                  {'yearAndMonth':'2016-02','profitability':298},
                  {'yearAndMonth':'2016-03','profitability':334},
                  {'yearAndMonth':'2016-04','profitability':128}]

products = [{'yearAndMonth':'2015-01','monthlyAverage':15559,'product':'Investment'},
            {'yearAndMonth':'2015-02','monthlyAverage':13163,'product':'Investment'},
            {'yearAndMonth':'2015-03','monthlyAverage':13306,'product':'Investment'},
            {'yearAndMonth':'2015-04','monthlyAverage':14998,'product':'Investment'},
            {'yearAndMonth':'2015-05','monthlyAverage':13286,'product':'Investment'},
            {'yearAndMonth':'2015-06','monthlyAverage':15875,'product':'Investment'},
            {'yearAndMonth':'2015-07','monthlyAverage':15444,'product':'Investment'},
            {'yearAndMonth':'2015-08','monthlyAverage':15246,'product':'Investment'},
            {'yearAndMonth':'2015-09','monthlyAverage':15361,'product':'Investment'},
            {'yearAndMonth':'2015-10','monthlyAverage':17211,'product':'Investment'},
            {'yearAndMonth':'2015-11','monthlyAverage':14757,'product':'Investment'},
            {'yearAndMonth':'2015-12','monthlyAverage':15594,'product':'Investment'},
            {'yearAndMonth':'2016-01','monthlyAverage':15752,'product':'Investment'},
            {'yearAndMonth':'2016-02','monthlyAverage':14033,'product':'Investment'},
            {'yearAndMonth':'2016-03','monthlyAverage':15198,'product':'Investment'},
            {'yearAndMonth':'2016-04','monthlyAverage':11246,'product':'Investment'},
            {'yearAndMonth':'2015-01','monthlyAverage':8929,'product':'Savings'},
            {'yearAndMonth':'2015-02','monthlyAverage':8253,'product':'Savings'},
            {'yearAndMonth':'2015-03','monthlyAverage':7930,'product':'Savings'},
            {'yearAndMonth':'2015-04','monthlyAverage':8320,'product':'Savings'},
            {'yearAndMonth':'2015-05','monthlyAverage':7999,'product':'Savings'},
            {'yearAndMonth':'2015-06','monthlyAverage':8414,'product':'Savings'},
            {'yearAndMonth':'2015-07','monthlyAverage':7362,'product':'Savings'},
            {'yearAndMonth':'2015-08','monthlyAverage':8025,'product':'Savings'},
            {'yearAndMonth':'2015-09','monthlyAverage':7258,'product':'Savings'},
            {'yearAndMonth':'2015-10','monthlyAverage':7380,'product':'Savings'},
            {'yearAndMonth':'2015-11','monthlyAverage':6672,'product':'Savings'},
            {'yearAndMonth':'2015-12','monthlyAverage':9615,'product':'Savings'},
            {'yearAndMonth':'2016-01','monthlyAverage':9978,'product':'Savings'},
            {'yearAndMonth':'2016-02','monthlyAverage':8529,'product':'Savings'},
            {'yearAndMonth':'2016-03','monthlyAverage':7684,'product':'Savings'},
            {'yearAndMonth':'2016-04','monthlyAverage':7374,'product':'Savings'},
            {'yearAndMonth':'2015-01','monthlyAverage':1010*5,'product':'Checking'},
            {'yearAndMonth':'2015-02','monthlyAverage': 853*5,'product':'Checking'},
            {'yearAndMonth':'2015-03','monthlyAverage': 957*5,'product':'Checking'},
            {'yearAndMonth':'2015-04','monthlyAverage':1002*5,'product':'Checking'},
            {'yearAndMonth':'2015-05','monthlyAverage':1005*5,'product':'Checking'},
            {'yearAndMonth':'2015-06','monthlyAverage': 747*5,'product':'Checking'},
            {'yearAndMonth':'2015-07','monthlyAverage': 959*5,'product':'Checking'},
            {'yearAndMonth':'2015-08','monthlyAverage':1141*5,'product':'Checking'},
            {'yearAndMonth':'2015-09','monthlyAverage': 810*5,'product':'Checking'},
            {'yearAndMonth':'2015-10','monthlyAverage':1227*5,'product':'Checking'},
            {'yearAndMonth':'2015-11','monthlyAverage':1009*5,'product':'Checking'},
            {'yearAndMonth':'2015-12','monthlyAverage':1044*5,'product':'Checking'},
            {'yearAndMonth':'2016-01','monthlyAverage':1112*5,'product':'Checking'},
            {'yearAndMonth':'2016-02','monthlyAverage':1028*5,'product':'Checking'},
            {'yearAndMonth':'2016-03','monthlyAverage':1095*5,'product':'Checking'},
            {'yearAndMonth':'2016-04','monthlyAverage':1054*5,'product':'Checking'}]

In [37]:
monthly_profit = [{'date':'2015-01','profitability':303},
                  {'date':'2015-02','profitability':340},
                  {'date':'2015-03','profitability':295},
                  {'date':'2015-04','profitability':293},
                  {'date':'2015-05','profitability':292},
                  {'date':'2015-06','profitability':267},
                  {'date':'2015-07','profitability':348},
                  {'date':'2015-08','profitability':327},
                  {'date':'2015-09','profitability':261},
                  {'date':'2015-10','profitability':257},
                  {'date':'2015-11','profitability':325},
                  {'date':'2015-12','profitability':332},
                  {'date':'2016-01','profitability':277},
                  {'date':'2016-02','profitability':298},
                  {'date':'2016-03','profitability':334},
                  {'date':'2016-04','profitability':128}]

In [41]:
bank_data = pd.read_csv('data/bank-full.csv',';')

ref_id = np.random.choice(np.arange(len(bank_data)), customern)

def sample_table(ref_id_i, name_i):
    row = bank_data.loc[ref_id_i]
    maritalStatus = row['marital']
    investableAssetEstimate = np.round(max(row['balance']*100+np.random.randn()*10000,0),-3)
    creditScoreFICO = np.round(np.random.rand()*300+400,-1)
    email = name_i.lower().replace(' ','.')+'@mail.com'
    numberOfChildren = 0 if row['marital'] == 'single' else np.round((np.random.randn()/1.8)**2)
    zillowHomeValueEstimation = 0 if row['housing']=='no' else np.round(150000+np.random.randn()*10000,-4)
    homeEquityEstimate = np.round(zillowHomeValueEstimation * (np.random.rand()*0.75+0.25),-4)
    housing = row['housing']
    age = row['age']
    nbo_probs = (np.random.rand(4)**2+np.random.rand(4)**2)/4
    sign_rev = np.random.rand(4)>0.125
    nbo_probs[sign_rev] = 1-nbo_probs[sign_rev]

    nextBestOffers = [{"nboName":"Platinum Credit Card","nboProbability":nbo_probs[0]},
                      {"nboName":"Shopper Rewarding System","nboProbability":nbo_probs[1]}, 
                      {"nboName":"Advanced Investment Account","nboProbability":nbo_probs[2]},
                      {"nboName":"Student Checking Account","nboProbability":nbo_probs[3]}]
    return {
        'maritalStatus':maritalStatus,
        'investableAssetEstimate':investableAssetEstimate,
        'creditScoreFICO':creditScoreFICO,
        'email':email,
        'numberOfChildren':numberOfChildren,
        'zillowHomeValueEstimation':zillowHomeValueEstimation,
        'homeEquityEstimate':homeEquityEstimate,
        'housing':housing,
        'age':age,
        'nextBestOffers':nextBestOffers
    }

customer_id = range(1,customern+1)

customers = [{'customerId':customer_id_i,
              'name':name_i,
              'gender':gender_i, 
              'state':state_i,
              'status':status_i,
              'churn_rate':churn_rate_i,
              'monthlyProfitabilities':monthly_profit,
              'products':products,
              'surveys':surveys_d_i
             }
             for customer_id_i, name_i, gender_i, state_i, status_i, churn_rate_i, surveys_d_i
             in zip(customer_id, name, gender, state, status, churn_rate, surveys_d)]

customers2 = [dict(c, **(sample_table(ref_id_i, c['name']))) for c,ref_id_i in zip(customers,ref_id)]

db.customers.insert_many(customers2)

In [45]:
360/7.*np.arange(7)-90

array([ -90.        ,  -38.57142857,   12.85714286,   64.28571429,
        115.71428571,  167.14285714,  218.57142857])

In [42]:
activities = [{
'activityType':'complaint',
'activityChannel':'call',
'date':'2016-03-04',
'customerId':1
},{
'activityType':'complaint',
'activityChannel':'call',
'date':'2016-03-12',
'customerId':1
},{
'activityType':'inquiry',
'activityChannel':'call',
'date':'2016-02-11',
'customerId':1
},{
'activityType':'complaint',
'activityChannel':'call',
'date':'2016-01-01',
'customerId':1
},{
'activityType':'complaint',
'activityChannel':'e-mail',
'date':'2016-02-07',
'customerId':1,
'sentiment':-2
},{
'activityType':'complaint',
'activityChannel':'twitter',
'date':'2016-04-01',
'customerId':1,
'sentiment':-3
}]

db.activities.insert_many(activities)